# Predicting Major Flight Delays

Our client, FlightChicken, would like a model that predicts whether a flight will experience a major delay. Delays can cause a major disruption to travel plans, especially if they cause a person to miss their connecting flight. FlightChicken would like to give their users a heads up about potential travel disruptions like this.

This is a major undertaking as there are hundreds of airlines and thousands of airports in the United States alone. That's why FlightChicken would like to launch with just an MVP to prove our their concept. This MVP should support major US airports and 8 of the most popular airlines.

In [1]:
import pandas as pd
import glob
import os
import requests
import json
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

import airportsdata
from pytz import timezone
import pytz

## Business Understanding

MVP should support:

* [Top 8 US Airlines](https://www.statista.com/statistics/250577/domestic-market-share-of-leading-us-airlines/)
 * American Airlines
 * Delta Air Lines
 * United Airlines
 * Southwest Airlines
 * Alaska Airlines
 * JetBlue Airways
 * Spirit
 * SkyWest
* [Large and medium airport hubs](https://www.faa.gov/airports/planning_capacity/passenger_allcargo_stats/passenger/media/cy20-commercial-service-enplanements.pdf)
 * "The term hub is used by the FAA to identify very busy commercial service airports. Large hubs are the airports that each account for at least one percent of total U.S. passenger enplanements."
 * In 2020 these accounted for 84% of all enplanements


## Data

To complete this project, we will be using data from several sources.

1. **Bureau of Transportation Statistics: Carrier On-Time Performence Database.** This database contains scheduled and actual departure and arrival times reported by certified U.S. air carriers that account for at least one percent of domestic scheduled passenger revenues. The data is collected by the Office of Airline Information, Bureau of Transportation Statistics (BTS).
2. **National Oceanic and Atmospheric Administration (NOAA): Daily Weather Summaries:** Data on select weather conditions at airprots collected by weather stations.
3. **Timezone for Each Airport by StackOverflow user hroptatyr:** This data will allow us to convert our timedata to UTC and make it easier to work with. [Link](https://raw.githubusercontent.com/hroptatyr/dateutils/tzmaps/iata.tzmap)

Additionally, to link data from NOAA to each airport, I manually looked up the weather station for all airports relevant in this project. This data was can be found in FILE PATH. To reproduce, go to [Climate Data Online Search](https://www.ncei.noaa.gov/cdo-web/search) and make the following selections:

1. Select Weather Observation Type/Dataset: Daily Summaries
2. Select Date Range: 2018-01-01 to 2021-12-31
3. Search For: Stations
4. Enter a Search Term: enter the city and state of the airport plus the term 'airport'. e.g. Atlanta, GA airport
5. Hit 'Search'
6. On the results page, find the closest/most relevant weather station. In the example of "Atlanta, GA airport" you would select 'Atlanta Hartsfield Jackson International Airport". Hit 'Add to Cart'. **On the results page, make note of the Station ID. This is what will serve as the key for linking BTS data with weather data.**
7. Repeat for every airport.


### Data Preperation & Cleaning
First, we need to load all our data into pandas so that we can work with it.

#### Carrier On-Time Performence Database
This data can only be downloaded by month, which means it is split among many files.

In [2]:
flight_data = glob.glob(os.path.join('data/downloaded/carrier-on-time-performence', "*.csv"))
carrier_data = pd.concat((pd.read_csv(f) for f in flight_data), ignore_index=True)

<ipython-input-2-68625596303d>:2: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  carrier_data = pd.concat((pd.read_csv(f) for f in flight_data), ignore_index=True)


#### Timezone Data
Next, we bring in timezone data and merge it into our carrier data from above.

In [3]:
timezones = pd.read_csv('data/downloaded/timezones-for-each-airport.csv')
carrier_data['ORIGIN_TIMEZONE'] = carrier_data['ORIGIN'].map(timezones.set_index('ORIGIN')['timezone'])

In [4]:
# Check whether we were able to match all records
carrier_data['ORIGIN_TIMEZONE'].isna().sum()

2428

Looks like 2,428 records couldn't be matched to a timezone. That's not bad considering we have 25,115,464 records. We'll simply drop the Null values.

In [5]:
carrier_data = carrier_data[carrier_data['ORIGIN_TIMEZONE'].notna()]

The time feature we are concerned with is takeoff time.  Our goal is to convert it to a universal UTC time. To do this, we first need to transform it a bit so that it's workable.

In [6]:
carrier_data[['FL_DATE', 'CRS_DEP_TIME']]

,FL_DATE,CRS_DEP_TIME
0,2/26/2018 12:00:00 AM,940
1,2/26/2018 12:00:00 AM,745
2,2/26/2018 12:00:00 AM,1814
3,2/26/2018 12:00:00 AM,1040
4,2/26/2018 12:00:00 AM,1504
...,...,...
25115459,7/22/2019 12:00:00 AM,1500
25115460,7/23/2019 12:00:00 AM,1500
25115461,7/24/2019 12:00:00 AM,1500
25115462,7/25/2019 12:00:00 AM,1500


All times are expressed as an integer in military time. For example, 940 is 9:40am and 1500 is 3:00pm. We would like to first convert it to a string that can be read as 24H time, then combined with the FL_DATE field so that we can have an exact take-off date and time.

In [7]:
# First we create a helper function to carry out the transformation

def float_to_time(time):
    '''
    Function takes in an integer representation of time (24-hour format)
    and returns a string in proper datetime formatting. Example: 1545 (int) becomes 15:45 (string)
    '''
    time_str = str(time)
    digits = len(time_str)
    if time_str == 
    if digits < 2:
        return '00:0' + str(time)
    if digits == 2:
        return '00:' + str(time)
    if digits == 3:
        return '0' + time_str[:1] + ':' + time_str[1:]
    if digits == 4:
        return time_str[:2] + ':' + time_str[2:]

In [8]:
# First, we apply the function above to transform the CRS_DEP_TIME field
carrier_data['CRS_DEP_TIME'] = carrier_data['CRS_DEP_TIME'].apply(float_to_time)
# Next, we update the FL_DATE field so that it now contains the proper date AND time of takeoff
carrier_data['FL_DATE'] =  pd.to_datetime(carrier_data['FL_DATE'].astype(str) + ' ' + carrier_data['CRS_DEP_TIME'])

Now that we have the above expressed in LOCAL time, we will use the timezone data to create an additional element in UTC time.

In [9]:
# First we need to make our data, which is timezone naive, to timezone aware
carrier_data['FL_DATE'] = carrier_data['FL_DATE'].astype('datetime64[ns]')
carrier_data['FL_DATE'] = carrier_data.apply(lambda x: x['FL_DATE'].replace(tzinfo=timezone(x['ORIGIN_TIMEZONE'])), axis=1)

In [10]:
# Next, we convert the timezone
carrier_data['FL_DATE_UTC'] = carrier_data.apply(lambda x: x['FL_DATE'].tz_convert(pytz.utc), axis=1)

### Airport Congestion
One hypothesis is that flight delays can be tied to airport "traffic" (or congestion). It stands to reason that if an airline has 100 flights scheduled to take off at 11am, there is a higher chance of delays than if there are only 10 flights scheduled to take off.

Moreover, because traffic and delays in the morning can propegate throughout the day congestion before our flight takes off can also play a role.

We'll create a set of features that put a number on this congestion.

First, we will round takeoff times to the nearest hour. This will make calculations easier. Then, we will calculate average flights on a paritcular date and time, as well as the average amount of flights at different time intervals (1 hour before take off, 3 hours before takeoff, 6 hours before takeoff, 12 hours before takeoff and 24 hours before takeoff).

In [11]:
def takeoff_hour_rounder(time):
    '''
    Function takes in a time and returns time rounded to the 
    nearest hour by adding a timedelta hour if minute >= 30
    '''
    return (time.replace(second=0, microsecond=0, minute=0, hour=time.hour)
               +timedelta(hours=time.minute//30))

In [12]:
carrier_data['FL_DATE_UTC'] =  pd.to_datetime(carrier_data['FL_DATE_UTC'], utc=True)
carrier_data['FL_DATE_UTC_ROUNDED'] = carrier_data['FL_DATE_UTC'].apply(takeoff_hour_rounder)
carrier_data['FL_DATE_UTC_ROUNDED'] =  pd.to_datetime(carrier_data['FL_DATE_UTC_ROUNDED'])

In [13]:
# Next we create a key for calculating congestion at each airport
carrier_data['congestion-by-hour-key'] = carrier_data['ORIGIN'] \
                        + carrier_data['FL_DATE_UTC_ROUNDED'].dt.month.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED'].dt.day.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED'].dt.hour.astype(str)

In [14]:
# Now we create a new dataframe that holds data on congestion
airport_congestion_by_hour = carrier_data.groupby('congestion-by-hour-key')['TAIL_NUM'].count()
airport_congestion_by_hour = airport_congestion_by_hour.to_frame()
airport_congestion_by_hour.reset_index(inplace=True)
airport_congestion_by_hour.rename(columns={'TAIL_NUM': 'count_of_flights'}, inplace=True)

In [15]:
# Because we are counting by multiple years, we also create a dataframe
# that shows how many years are in our data to calculate an average
years_covered = carrier_data.groupby('congestion-by-hour-key')['YEAR'].nunique()
years_covered = years_covered.to_frame()
years_covered.reset_index(inplace=True)
years_covered.rename(columns={'YEAR': 'years_covered'}, inplace=True)

In [16]:
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_1'] = carrier_data['FL_DATE_UTC_ROUNDED'] - timedelta(hours=1, minutes=0)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_2'] = carrier_data['FL_DATE_UTC_ROUNDED'] - timedelta(hours=2, minutes=0)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_3'] = carrier_data['FL_DATE_UTC_ROUNDED'] - timedelta(hours=3, minutes=0)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_4'] = carrier_data['FL_DATE_UTC_ROUNDED'] - timedelta(hours=4, minutes=0)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_5'] = carrier_data['FL_DATE_UTC_ROUNDED'] - timedelta(hours=5, minutes=0)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_6'] = carrier_data['FL_DATE_UTC_ROUNDED'] - timedelta(hours=6, minutes=0)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_7'] = carrier_data['FL_DATE_UTC_ROUNDED'] - timedelta(hours=7, minutes=0)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_8'] = carrier_data['FL_DATE_UTC_ROUNDED'] - timedelta(hours=8, minutes=0)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_9'] = carrier_data['FL_DATE_UTC_ROUNDED'] - timedelta(hours=9, minutes=0)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_10'] = carrier_data['FL_DATE_UTC_ROUNDED'] - timedelta(hours=10, minutes=0)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_11'] = carrier_data['FL_DATE_UTC_ROUNDED'] - timedelta(hours=11, minutes=0)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_12'] = carrier_data['FL_DATE_UTC_ROUNDED'] - timedelta(hours=12, minutes=0)

In [17]:
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_1_key'] = carrier_data['ORIGIN'] \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_1'].dt.month.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_1'].dt.day.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_1'].dt.hour.astype(str)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_2_key'] = carrier_data['ORIGIN'] \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_2'].dt.month.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_2'].dt.day.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_2'].dt.hour.astype(str)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_3_key'] = carrier_data['ORIGIN'] \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_3'].dt.month.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_3'].dt.day.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_3'].dt.hour.astype(str)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_4_key'] = carrier_data['ORIGIN'] \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_4'].dt.month.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_4'].dt.day.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_4'].dt.hour.astype(str)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_5_key'] = carrier_data['ORIGIN'] \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_5'].dt.month.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_5'].dt.day.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_5'].dt.hour.astype(str)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_6_key'] = carrier_data['ORIGIN'] \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_6'].dt.month.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_6'].dt.day.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_6'].dt.hour.astype(str)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_7_key'] = carrier_data['ORIGIN'] \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_7'].dt.month.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_7'].dt.day.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_7'].dt.hour.astype(str)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_8_key'] = carrier_data['ORIGIN'] \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_8'].dt.month.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_8'].dt.day.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_8'].dt.hour.astype(str)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_9_key'] = carrier_data['ORIGIN'] \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_9'].dt.month.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_9'].dt.day.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_9'].dt.hour.astype(str)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_10_key'] = carrier_data['ORIGIN'] \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_10'].dt.month.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_10'].dt.day.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_10'].dt.hour.astype(str)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_11_key'] = carrier_data['ORIGIN'] \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_11'].dt.month.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_11'].dt.day.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_11'].dt.hour.astype(str)
carrier_data['FL_DATE_UTC_ROUNDED_Tminus_12_key'] = carrier_data['ORIGIN'] \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_12'].dt.month.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_12'].dt.day.astype(str) \
                        + carrier_data['FL_DATE_UTC_ROUNDED_Tminus_12'].dt.hour.astype(str)

In [18]:
# We won't be using the rounded times again, so it's good to just drop those columns for the sake of memory
carrier_data.drop(columns=['FL_DATE_UTC_ROUNDED', 'FL_DATE_UTC_ROUNDED_Tminus_1', 'FL_DATE_UTC_ROUNDED_Tminus_2',
                 'FL_DATE_UTC_ROUNDED_Tminus_3', 'FL_DATE_UTC_ROUNDED_Tminus_4',
                 'FL_DATE_UTC_ROUNDED_Tminus_5', 'FL_DATE_UTC_ROUNDED_Tminus_6',
                 'FL_DATE_UTC_ROUNDED_Tminus_7', 'FL_DATE_UTC_ROUNDED_Tminus_8',
                 'FL_DATE_UTC_ROUNDED_Tminus_9', 'FL_DATE_UTC_ROUNDED_Tminus_10',
                 'FL_DATE_UTC_ROUNDED_Tminus_11', 'FL_DATE_UTC_ROUNDED_Tminus_12'], inplace=True)

In [19]:
carrier_data['CONGESTION_TMINUS_0'] = carrier_data['congestion-by-hour-key'].map(airport_congestion_by_hour.set_index('congestion-by-hour-key')['count_of_flights'])
airport_congestion_by_hour.columns = airport_congestion_by_hour.columns.str.replace("congestion-by-hour-key", "FL_DATE_UTC_ROUNDED_Tminus_1_key")
carrier_data['CONGESTION_TMINUS_1'] = carrier_data['FL_DATE_UTC_ROUNDED_Tminus_1_key'].map(airport_congestion_by_hour.set_index('FL_DATE_UTC_ROUNDED_Tminus_1_key')['count_of_flights'])
airport_congestion_by_hour.columns = airport_congestion_by_hour.columns.str.replace("FL_DATE_UTC_ROUNDED_Tminus_1_key", "FL_DATE_UTC_ROUNDED_Tminus_2_key")
carrier_data['CONGESTION_TMINUS_2'] = carrier_data['FL_DATE_UTC_ROUNDED_Tminus_2_key'].map(airport_congestion_by_hour.set_index('FL_DATE_UTC_ROUNDED_Tminus_2_key')['count_of_flights'])
airport_congestion_by_hour.columns = airport_congestion_by_hour.columns.str.replace("FL_DATE_UTC_ROUNDED_Tminus_2_key", "FL_DATE_UTC_ROUNDED_Tminus_3_key")
carrier_data['CONGESTION_TMINUS_3'] = carrier_data['FL_DATE_UTC_ROUNDED_Tminus_3_key'].map(airport_congestion_by_hour.set_index('FL_DATE_UTC_ROUNDED_Tminus_3_key')['count_of_flights'])
airport_congestion_by_hour.columns = airport_congestion_by_hour.columns.str.replace("FL_DATE_UTC_ROUNDED_Tminus_3_key", "FL_DATE_UTC_ROUNDED_Tminus_4_key")
carrier_data['CONGESTION_TMINUS_4'] = carrier_data['FL_DATE_UTC_ROUNDED_Tminus_4_key'].map(airport_congestion_by_hour.set_index('FL_DATE_UTC_ROUNDED_Tminus_4_key')['count_of_flights'])
airport_congestion_by_hour.columns = airport_congestion_by_hour.columns.str.replace("FL_DATE_UTC_ROUNDED_Tminus_4_key", "FL_DATE_UTC_ROUNDED_Tminus_5_key")
carrier_data['CONGESTION_TMINUS_5'] = carrier_data['FL_DATE_UTC_ROUNDED_Tminus_5_key'].map(airport_congestion_by_hour.set_index('FL_DATE_UTC_ROUNDED_Tminus_5_key')['count_of_flights'])
airport_congestion_by_hour.columns = airport_congestion_by_hour.columns.str.replace("FL_DATE_UTC_ROUNDED_Tminus_5_key", "FL_DATE_UTC_ROUNDED_Tminus_6_key")
carrier_data['CONGESTION_TMINUS_6'] = carrier_data['FL_DATE_UTC_ROUNDED_Tminus_6_key'].map(airport_congestion_by_hour.set_index('FL_DATE_UTC_ROUNDED_Tminus_6_key')['count_of_flights'])
airport_congestion_by_hour.columns = airport_congestion_by_hour.columns.str.replace("FL_DATE_UTC_ROUNDED_Tminus_6_key", "FL_DATE_UTC_ROUNDED_Tminus_7_key")
carrier_data['CONGESTION_TMINUS_7'] = carrier_data['FL_DATE_UTC_ROUNDED_Tminus_7_key'].map(airport_congestion_by_hour.set_index('FL_DATE_UTC_ROUNDED_Tminus_7_key')['count_of_flights'])
airport_congestion_by_hour.columns = airport_congestion_by_hour.columns.str.replace("FL_DATE_UTC_ROUNDED_Tminus_7_key", "FL_DATE_UTC_ROUNDED_Tminus_8_key")
carrier_data['CONGESTION_TMINUS_8'] = carrier_data['FL_DATE_UTC_ROUNDED_Tminus_8_key'].map(airport_congestion_by_hour.set_index('FL_DATE_UTC_ROUNDED_Tminus_8_key')['count_of_flights'])
airport_congestion_by_hour.columns = airport_congestion_by_hour.columns.str.replace("FL_DATE_UTC_ROUNDED_Tminus_8_key", "FL_DATE_UTC_ROUNDED_Tminus_9_key")
carrier_data['CONGESTION_TMINUS_9'] = carrier_data['FL_DATE_UTC_ROUNDED_Tminus_9_key'].map(airport_congestion_by_hour.set_index('FL_DATE_UTC_ROUNDED_Tminus_9_key')['count_of_flights'])
airport_congestion_by_hour.columns = airport_congestion_by_hour.columns.str.replace("FL_DATE_UTC_ROUNDED_Tminus_9_key", "FL_DATE_UTC_ROUNDED_Tminus_10_key")
carrier_data['CONGESTION_TMINUS_10'] = carrier_data['FL_DATE_UTC_ROUNDED_Tminus_10_key'].map(airport_congestion_by_hour.set_index('FL_DATE_UTC_ROUNDED_Tminus_10_key')['count_of_flights'])
airport_congestion_by_hour.columns = airport_congestion_by_hour.columns.str.replace("FL_DATE_UTC_ROUNDED_Tminus_10_key", "FL_DATE_UTC_ROUNDED_Tminus_11_key")
carrier_data['CONGESTION_TMINUS_11'] = carrier_data['FL_DATE_UTC_ROUNDED_Tminus_11_key'].map(airport_congestion_by_hour.set_index('FL_DATE_UTC_ROUNDED_Tminus_11_key')['count_of_flights'])
airport_congestion_by_hour.columns = airport_congestion_by_hour.columns.str.replace("FL_DATE_UTC_ROUNDED_Tminus_11_key", "FL_DATE_UTC_ROUNDED_Tminus_12_key")
carrier_data['CONGESTION_TMINUS_12'] = carrier_data['FL_DATE_UTC_ROUNDED_Tminus_12_key'].map(airport_congestion_by_hour.set_index('FL_DATE_UTC_ROUNDED_Tminus_12_key')['count_of_flights'])

In [20]:
carrier_data['years_covered'] = carrier_data['congestion-by-hour-key'].map(years_covered.set_index('congestion-by-hour-key')['years_covered'])

In [21]:
carrier_data['CONGESTION_TMINUS_0'] = carrier_data['CONGESTION_TMINUS_0'].fillna(0)
carrier_data['CONGESTION_TMINUS_1'] = carrier_data['CONGESTION_TMINUS_1'].fillna(0)
carrier_data['CONGESTION_TMINUS_2'] = carrier_data['CONGESTION_TMINUS_2'].fillna(0)
carrier_data['CONGESTION_TMINUS_3'] = carrier_data['CONGESTION_TMINUS_3'].fillna(0)
carrier_data['CONGESTION_TMINUS_4'] = carrier_data['CONGESTION_TMINUS_4'].fillna(0)
carrier_data['CONGESTION_TMINUS_5'] = carrier_data['CONGESTION_TMINUS_5'].fillna(0)
carrier_data['CONGESTION_TMINUS_6'] = carrier_data['CONGESTION_TMINUS_6'].fillna(0)
carrier_data['CONGESTION_TMINUS_7'] = carrier_data['CONGESTION_TMINUS_7'].fillna(0)
carrier_data['CONGESTION_TMINUS_8'] = carrier_data['CONGESTION_TMINUS_8'].fillna(0)
carrier_data['CONGESTION_TMINUS_9'] = carrier_data['CONGESTION_TMINUS_9'].fillna(0)
carrier_data['CONGESTION_TMINUS_10'] = carrier_data['CONGESTION_TMINUS_10'].fillna(0)
carrier_data['CONGESTION_TMINUS_11'] = carrier_data['CONGESTION_TMINUS_11'].fillna(0)
carrier_data['CONGESTION_TMINUS_12'] = carrier_data['CONGESTION_TMINUS_12'].fillna(0)

In [22]:
# We won't be using the these keys again, so it's good to just drop those columns for the sake of memory
carrier_data.drop(columns=['congestion-by-hour-key', 'FL_DATE_UTC_ROUNDED_Tminus_1_key', 'FL_DATE_UTC_ROUNDED_Tminus_2_key',
                 'FL_DATE_UTC_ROUNDED_Tminus_3_key', 'FL_DATE_UTC_ROUNDED_Tminus_4_key',
                 'FL_DATE_UTC_ROUNDED_Tminus_5_key', 'FL_DATE_UTC_ROUNDED_Tminus_6_key',
                 'FL_DATE_UTC_ROUNDED_Tminus_7_key', 'FL_DATE_UTC_ROUNDED_Tminus_8_key',
                 'FL_DATE_UTC_ROUNDED_Tminus_9_key', 'FL_DATE_UTC_ROUNDED_Tminus_10_key',
                 'FL_DATE_UTC_ROUNDED_Tminus_11_key', 'FL_DATE_UTC_ROUNDED_Tminus_12_key'], inplace=True)

In [23]:
carrier_data['CONGESTION_TMINUS_0-3'] = carrier_data['CONGESTION_TMINUS_0'] \
                                    + carrier_data['CONGESTION_TMINUS_1'] \
                                    + carrier_data['CONGESTION_TMINUS_2'] \
                                    + carrier_data['CONGESTION_TMINUS_3'] 

carrier_data['CONGESTION_TMINUS_0-6'] = carrier_data['CONGESTION_TMINUS_0'] \
                                    + carrier_data['CONGESTION_TMINUS_1'] \
                                    + carrier_data['CONGESTION_TMINUS_2'] \
                                    + carrier_data['CONGESTION_TMINUS_3'] \
                                    + carrier_data['CONGESTION_TMINUS_4'] \
                                    + carrier_data['CONGESTION_TMINUS_5'] \
                                    + carrier_data['CONGESTION_TMINUS_6']

carrier_data['CONGESTION_TMINUS_0-12'] = carrier_data['CONGESTION_TMINUS_0'] \
                                    + carrier_data['CONGESTION_TMINUS_1'] \
                                    + carrier_data['CONGESTION_TMINUS_2'] \
                                    + carrier_data['CONGESTION_TMINUS_3'] \
                                    + carrier_data['CONGESTION_TMINUS_4'] \
                                    + carrier_data['CONGESTION_TMINUS_5'] \
                                    + carrier_data['CONGESTION_TMINUS_6'] \
                                    + carrier_data['CONGESTION_TMINUS_7'] \
                                    + carrier_data['CONGESTION_TMINUS_8'] \
                                    + carrier_data['CONGESTION_TMINUS_9'] \
                                    + carrier_data['CONGESTION_TMINUS_10'] \
                                    + carrier_data['CONGESTION_TMINUS_11'] \
                                    + carrier_data['CONGESTION_TMINUS_12']

In [24]:
carrier_data['AVG_CONGESTION_TMINUS_0'] = carrier_data['CONGESTION_TMINUS_0'] / carrier_data['years_covered']
carrier_data['AVG_CONGESTION_TMINUS_0-3'] = carrier_data['CONGESTION_TMINUS_0-3'] / carrier_data['years_covered']
carrier_data['AVG_CONGESTION_TMINUS_0-6'] = carrier_data['CONGESTION_TMINUS_0-6'] / carrier_data['years_covered']
carrier_data['AVG_CONGESTION_TMINUS_0-12'] = carrier_data['CONGESTION_TMINUS_0-12'] / carrier_data['years_covered']

In [25]:
# Again let's drop columns we won't need
carrier_data.drop(columns=['CONGESTION_TMINUS_0', 'CONGESTION_TMINUS_1', 'CONGESTION_TMINUS_2',
                          'CONGESTION_TMINUS_3', 'CONGESTION_TMINUS_4', 'CONGESTION_TMINUS_5',
                          'CONGESTION_TMINUS_6', 'CONGESTION_TMINUS_7', 'CONGESTION_TMINUS_8',
                          'CONGESTION_TMINUS_9', 'CONGESTION_TMINUS_10', 'CONGESTION_TMINUS_11', 
                          'CONGESTION_TMINUS_12', 'CONGESTION_TMINUS_0-3', 'CONGESTION_TMINUS_0-6',
                          'CONGESTION_TMINUS_0-12', 'years_covered'], inplace=True)

#### Filtering for Airports & Airlines Relevant to Business Case
The MVP calls for us to support flights originating from major US airports and 8 major airlines. So we filter down our data for that.

In [26]:
# Create list of relevant aiports based on business case
relevant_airports = ['ATL', 'DFW', 'DEN', 'ORD', 'LAX', 'CLT', 'LAS', 'PHX', 
                     'MCO', 'SEA', 'MIA', 'IAH', 'JFK', 'FLL', 'EWR', 'SFO', 'MSP', 'DTW',
                     'BOS', 'SLC', 'PHL', 'BWI', 'TPA', 'SAN', 'MDW', 'LGA', 'BNA', 'IAD',
                     'DAL', 'DCA', 'PDX', 'AUS', 'HOU', 'HNL', 'STL', 'RSW', 'SMF', 'MSY',
                     'SJU', 'RDU', 'OAK', 'MCI', 'CLE', 'IND', 'SAT', 'SNA', 'PIT', 'CVG',
                     'CMH', 'PBI', 'JAX', 'MKE', 'ONT', 'ANC', 'BDL', 'OGG', 'OMA', 'MEM',
                     'BOI', 'RNO', 'CHS', 'OKC']

# Create list of relevant IATA airline designators based on business case
relevant_airlines = ['WN', # Southwest
                     'DL', # Delta
                     'OO', # SkyWest
                     'AA', # American Airlines
                     'UA', # United Airlines
                     'B6', # JetBlue
                     'AS', # Alaska Airlines
                     'NK', # Spirit Airlines
                    ]

# Filter Dataframe to include only relevant airlines & airports
airport_filter = '|'.join(relevant_airports)
airline_filter = '|'.join(relevant_airlines)

carrier_data = carrier_data[carrier_data['ORIGIN'].str.contains(airport_filter)]
carrier_data = carrier_data[carrier_data['OP_CARRIER'].str.contains(airline_filter)]

#### Mering in Weather Data

In [27]:
# Importing weather station data
weather_df = pd.read_csv('data/downloaded/weather-data.csv')

# Importing data that maps each airport to a weather station
stations_airport_df = pd.read_csv('data/downloaded/weather-station-codes-airport-key.csv')

C:\Users\Robert\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (13,25,31,43,45,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
# First, we need to add the Weather Station ID for each airport in our dataset
carrier_data['weather_station_origin'] = carrier_data['ORIGIN'].map(stations_airport_df.set_index('airport_code')['noaa_station'])
carrier_data['weather_station_destination'] = carrier_data['DEST'].map(stations_airport_df.set_index('airport_code')['noaa_station'])

In [29]:
# We only care about certain fields in our weather dataframe so we filter for those
relevant_weather_cols = [
'STATION',
'DATE',
'LATITUDE',
'LONGITUDE',
'AWND',
'PRCP',
'SNOW',
'TMIN',
'TMAX',
'ELEVATION',
'SNWD',
'WDF2',
'WSF2',
'WT01',
'WT02',
'WT03',
'WT04',
'WT06',
'WT08'
]

weather_df = weather_df[relevant_weather_cols].copy()
weather_df.fillna(0, inplace=True)

In [30]:
# Create a key to prepare merging in daily weather data for each airport
weather_df['key'] = weather_df["STATION"].astype(str) + weather_df["DATE"].astype(str)
carrier_data['key'] = carrier_data['weather_station_origin'] + carrier_data['YEAR'].astype(str) + '/' + carrier_data['MONTH'].astype(str).str.zfill(2) + '/' + carrier_data['DAY_OF_MONTH'].astype(str).str.zfill(2)
carrier_data = carrier_data.merge(weather_df, on='key')
weather_df = weather_df.add_prefix('dest_')
carrier_data['dest_key'] = carrier_data['weather_station_destination'] + carrier_data['YEAR'].astype(str) + '/' + carrier_data['MONTH'].astype(str).str.zfill(2) + '/' + carrier_data['DAY_OF_MONTH'].astype(str).str.zfill(2)
carrier_data = carrier_data.merge(weather_df, on='dest_key')

In [31]:
# Once again, we clean up columns we don't need anymore
carrier_data.drop(columns=['key', 'dest_key', 'weather_station_origin', 'weather_station_destination',
                          'STATION', 'dest_STATION', 'DATE', 'dest_DATE', 'dest_LATITUDE', 'dest_LONGITUDE',
                          'LATITUDE', 'LONGITUDE'], inplace=True)

#### Proximity to Holidays
Anyone who has ever traveled by plane knows that delays seem to be most prevelant around the holidays. That's why one last feature we want to engineer is some sort of proximity to holidays.

In [35]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

date_range = pd.date_range(start='2018-01-01', end='2021-12-31')
cal = calendar()
holidays = cal.holidays(start=date_range.min(), end=date_range.max(), return_name=True)
holidays.reset_index(name='holiday').rename(columns={'index':'date'})
holidays = holidays.to_frame()
holidays.reset_index(inplace=True)
holidays.columns = ['holiday_date', 'holiday_name']

In [36]:
holidays

,holiday_date,holiday_name
0,2018-01-01,New Years Day
1,2018-01-15,Martin Luther King Jr. Day
2,2018-02-19,Presidents Day
3,2018-05-28,Memorial Day
4,2018-07-04,July 4th
5,2018-09-03,Labor Day
6,2018-10-08,Columbus Day
7,2018-11-12,Veterans Day
8,2018-11-22,Thanksgiving
9,2018-12-25,Christmas


In [38]:
carrier_data['holiday-key'] = carrier_data['YEAR'].astype(str) + '-' + carrier_data['MONTH'].astype(str).str.zfill(2) + '-' + carrier_data['DAY_OF_MONTH'].astype(str).str.zfill(2)
carrier_data['holiday-key'] = pd.to_datetime(carrier_data['holiday-key'])
carrier_data.sort_values('holiday-key', inplace=True)
carrier_data = pd.merge_asof(carrier_data, holidays, left_on='holiday-key', right_on='holiday_date',
                       direction='nearest', tolerance=pd.Timedelta(days=7))

In [66]:
carrier_data['days-from-holiday'] = (carrier_data['holiday-key'] - carrier_data['holiday_date']).dt.days
carrier_data['days-from-holiday'] = carrier_data['days-from-holiday'].astype(str)
carrier_data['days-from-specific-holiday'] = carrier_data['holiday_name'] + '_' + carrier_data['days-from-holiday'].astype(str)

In [67]:
# Cleaning up the results a bit
carrier_data['days-from-specific-holiday'].fillna('no-close-holiday', inplace=True)

In [70]:
# Once again, we clean up columns we don't need anymore
carrier_data.drop(columns=['holiday-key', 'holiday_name', 'holiday_date', 'days-from-holiday', 'days-from-holiday'], inplace=True)

#### Time of day takoff & landing
Lastly, we will create a continuous variable that quantifies when throughout the day that flight takes off. This is to account that it's possible for delays to be more prevalent at certain points in the day. To do this, we'll create a variable that measures a flight's distance from midnight in minutes.

In [81]:
carrier_data['temp_time'] = pd.to_datetime(carrier_data['CRS_DEP_TIME'])
carrier_data['takeoff-mins-from-midnight'] = ((carrier_data['temp_time'] - carrier_data['temp_time'].dt.normalize()) / pd.Timedelta('1 minute')).astype(int)

In [107]:
carrier_data['CRS_ARR_TIME'] = carrier_data['CRS_ARR_TIME'].apply(float_to_time)
carrier_data['CRS_ARR_TIME'] = carrier_data['CRS_ARR_TIME'].replace({'24:00':'00:00'})
carrier_data['temp_time_dest'] = pd.to_datetime(carrier_data['CRS_ARR_TIME'], format='%H:%M')
carrier_data['landing-mins-from-midnight'] = ((carrier_data['temp_time_dest'] - carrier_data['temp_time_dest'].dt.normalize()) / pd.Timedelta('1 minute')).astype(int)

In [108]:
carrier_data.drop(columns=['temp_time', 'temp_time_dest'], inplace=True)

### Dropping Irrelevant Columns
Our downloaded data has a lof of columns. We won't need many of them for our analysis. So here we drop remaining irrelevant columns.

In [116]:
cols_to_drop = ['QUARTER',
'FL_DATE',
'OP_UNIQUE_CARRIER',
'TAIL_NUM',
'DEST_AIRPORT_ID',
'CRS_DEP_TIME',
'DEP_TIME',
'TAXI_OUT',
'WHEELS_OFF',
'TAXI_IN',
'CRS_ARR_TIME',
'ARR_TIME',
'ARR_DELAY',
'CANCELLATION_CODE',
'DIVERTED',
'ACTUAL_ELAPSED_TIME',
'AIR_TIME',
'FLIGHTS',
'FIRST_DEP_TIME',
'TOTAL_ADD_GTIME',
'LONGEST_ADD_GTIME',
'ORIGIN_TIMEZONE',
'FL_DATE_UTC']

carrier_data.drop(cols_to_drop, axis = 1, inplace=True)

### Cleaning up missing data
As a last step, I'll deal with any missing values.

In [3]:
carrier_data.isna().sum().sort_values()

YEAR                                 0
WDF2                                 0
WSF2                                 0
WT01                                 0
WT02                                 0
WT03                                 0
WT04                                 0
WT06                                 0
WT08                                 0
dest_AWND                            0
dest_PRCP                            0
dest_SNOW                            0
SNWD                                 0
dest_TMIN                            0
dest_ELEVATION                       0
dest_SNWD                            0
dest_WDF2                            0
dest_WSF2                            0
dest_WT01                            0
dest_WT02                            0
dest_WT03                            0
dest_WT04                            0
dest_WT06                            0
dest_WT08                            0
days-from-specific-holiday           0
dest_TMAX                

First, it looks like we have a few missing 'DEP_DELAY_NEW' and "ARR_DELAY_NEW" fields. However, these are just 0.02% of our total records. So it's easy enough to just drop those.

In [4]:
carrier_data = carrier_data.dropna(subset=['DEP_DELAY_NEW', 'ARR_DELAY_NEW', 'CRS_ELAPSED_TIME'])

Now let's take another look.

In [5]:
carrier_data.isna().sum().sort_values()

YEAR                                 0
WDF2                                 0
WSF2                                 0
WT01                                 0
WT02                                 0
WT03                                 0
WT04                                 0
WT06                                 0
WT08                                 0
dest_AWND                            0
dest_PRCP                            0
dest_SNOW                            0
SNWD                                 0
dest_TMIN                            0
dest_ELEVATION                       0
dest_SNWD                            0
dest_WDF2                            0
dest_WSF2                            0
dest_WT01                            0
dest_WT02                            0
dest_WT03                            0
dest_WT04                            0
dest_WT06                            0
dest_WT08                            0
days-from-specific-holiday           0
dest_TMAX                

It looks like the delay cause fields (CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY and LATE_AIRCRAFT_DELAY) have many missing values. Lucikily, these fields are not needed for modeling so we can proceed.

In [11]:
# Let's remove all the extra columns that were created for the purpose of engineering these features
cols_with_bad_data = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
carrier_data.drop(cols_with_bad_data, axis = 1, inplace=True)

In [12]:
carrier_data.to_csv('data/prepared/cleaned_data.csv', index=False)